In [1]:
import requests
import pandas as pd
import time
import simfin as sf
from simfin.names import *

In [2]:
sf.set_api_key('xxxxxxxxxxxxxxxxxxxx')
# Set the local directory where data-files are stored.
# The dir will be created if it does not already exist.
sf.set_data_dir('~/simfin_data/')

In [3]:
# Load the annual Income Statements for all companies in the US.
# The data is automatically downloaded if you don't have it already.
df = sf.load_balance(variant='quarterly', market='us')
df_balance = df.copy()

Dataset "us-balance-quarterly" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!


In [4]:
#indexing
df_balance = sf.load(dataset='balance', variant='quarterly', market='us',
              index=[TICKER, REPORT_DATE],
              parse_dates=[REPORT_DATE, PUBLISH_DATE, RESTATED_DATE])

Dataset "us-balance-quarterly" on disk (0 days old).
- Loading from disk ... Done!


In [6]:
#Droping not needed columns write inplace = true for permanent delete
to_drop = ['Currency', 'Shares (Diluted)']
df_balance.drop(columns=to_drop, inplace=True)

In [69]:
#define columns, id vars, value vars
id_vars = list(df_balance.columns)[ : 13]
id_vars.append((df_balance.columns)[[15, 18, 19, 22]])
id_vars

['SimFinId',
 'Currency',
 'Fiscal Year',
 'Publish Date',
 'Restated Date',
 'Shares (Basic)',
 'Revenue',
 'Cost of Revenue',
 'Gross Profit',
 'Operating Expenses',
 'Selling, General & Administrative',
 'Research & Development',
 'Depreciation & Amortization',
 Index(['Interest Expense, Net', 'Pretax Income (Loss)',
        'Income Tax (Expense) Benefit, Net', 'Net Income'],
       dtype='object')]

In [6]:
#Melting losses into Losses column
df_melted = df_balance.copy()
df_melted = pd.melt(df_melted, id_vars=['SimFinId', 'Currency','Fiscal period', 'Fiscal Year', 'Publish Date', 'Restated Date', \
    'Shares (Basic)', 'Revenue', 'Cost of Revenue', 'Gross Profit', 'Operating Expenses', 'Selling, General & Administrative', \
        'Research & Development', 'Depreciation & Amortization','Interest Expense, Net', 'Income Tax (Expense) Benefit, Net', 'Net Income'], value_vars=['Operating Income (Loss)', \
            'Non-Operating Income (Loss)','Pretax Income (Loss), Adj.','Pretax Income (Loss)', \
                'Abnormal Gains (Losses)','Income (Loss) from Continuing Operations','Net Extraordinary Gains (Losses)'], value_name='Losses', ignore_index=False)

In [8]:
df_balance.dtypes

SimFinId                                                    int64
Fiscal Year                                                 int64
Fiscal Period                                              object
Publish Date                                       datetime64[ns]
Restated Date                                      datetime64[ns]
Shares (Basic)                                            float64
Cash, Cash Equivalents & Short Term Investments           float64
Accounts & Notes Receivable                               float64
Inventories                                               float64
Total Current Assets                                      float64
Property, Plant & Equipment, Net                          float64
Long Term Investments & Receivables                       float64
Other Long Term Assets                                    float64
Total Noncurrent Assets                                   float64
Total Assets                                                int64
Payables &

In [79]:
#null count - optional
df_null = df_melted.copy()
id_vars = list(df_null.columns)[ : 23]
df_null['num nulls'] = df_null[id_vars].isnull().sum(axis=1)
df_null.head()

SimFinId  Fiscal Year Publish Date Restated Date  \
Ticker Report Date                                                     
A      2016-10-31      45846         2016   2016-12-20    2018-12-20   
       2017-10-31      45846         2017   2017-12-21    2018-12-20   
       2018-10-31      45846         2018   2018-12-20    2020-12-18   
       2019-10-31      45846         2019   2019-12-19    2021-12-17   
       2020-10-31      45846         2020   2020-12-18    2021-12-17   

                    Shares (Basic)       Revenue  Cost of Revenue  \
Ticker Report Date                                                  
A      2016-10-31      326000000.0  4.202000e+09    -2.005000e+09   
       2017-10-31      322000000.0  4.472000e+09    -2.063000e+09   
       2018-10-31      321000000.0  4.914000e+09    -2.234000e+09   
       2019-10-31      314000000.0  5.163000e+09    -2.358000e+09   
       2020-10-31      309000000.0  5.339000e+09    -2.502000e+09   

                    Gross Profit  Operating Expenses  \
Ticker Report Date                                     
A      2016-10-31   2.197000e+09       -1.582000e+09   
       2017-10-31   2.409000e+09       -1.568000e+09   
       2018-10-31   2.680000e+09       -1.776000e+09   
       2019-10-31   2.805000e+09       -1.864000e+09   
       2020-10-31   2.837000e+09       -1.991000e+09   

                    Selling, General & Administrative  Research & Development  \
Ticker Report Date                                                              
A      2016-10-31                       -1.253000e+09            -329000000.0   
       2017-10-31                       -1.229000e+09            -339000000.0   
       2018-10-31                       -1.389000e+09            -387000000.0   
       2019-10-31                       -1.460000e+09            -404000000.0   
       2020-10-31                       -1.496000e+09            -495000000.0   

                    Depreciation & Amortization  Interest Expense, Net  \
Ticker Report Date                                                       
A      2016-10-31                           NaN            -61000000.0   
       2017-10-31                           NaN            -57000000.0   
       2018-10-31                           NaN            -37000000.0   
       2019-10-31                           NaN            -38000000.0   
       2020-10-31                           NaN            -70000000.0   

                    Income Tax (Expense) Benefit, Net  Net Income  \
Ticker Report Date                                                  
A      2016-10-31                         -82000000.0   462000000   
       2017-10-31                        -119000000.0   684000000   
       2018-10-31                        -630000000.0   316000000   
       2019-10-31                         152000000.0  1071000000   
       2020-10-31                        -123000000.0   719000000   

                         Losses  num nulls  
Ticker Report Date                          
A      2016-10-31   615000000.0          1  
       2017-10-31   841000000.0          1  
       2018-10-31   904000000.0          1  
       2019-10-31   941000000.0          1  
       2020-10-31   846000000.0          1

In [9]:
#replace null values
df_balance = df_balance.fillna(0)
df_balance

SimFinId  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                     
A      2016-10-31      45846         2016            Q4   2016-12-20   
       2017-01-31      45846         2017            Q1   2017-03-08   
       2017-04-30      45846         2017            Q2   2017-06-06   
       2017-07-31      45846         2017            Q3   2017-09-06   
       2017-10-31      45846         2017            Q4   2017-12-21   
...                      ...          ...           ...          ...   
ZYXI   2020-09-30     171401         2020            Q3   2020-10-27   
       2020-12-31     171401         2020            Q4   2021-02-25   
       2021-03-31     171401         2021            Q1   2021-04-29   
       2021-06-30     171401         2021            Q2   2021-07-29   
       2021-09-30     171401         2021            Q3   2021-11-02   

                   Restated Date  Shares (Basic)  \
Ticker Report Date                                 
A      2016-10-31     2017-12-21     324000000.0   
       2017-01-31     2017-03-08     322000000.0   
       2017-04-30     2017-06-06     321000000.0   
       2017-07-31     2017-09-06     321000000.0   
       2017-10-31     2018-12-20     324000000.0   
...                          ...             ...   
ZYXI   2020-09-30     2020-10-27      37945600.0   
       2020-12-31     2022-03-22      38262400.0   
       2021-03-31     2021-04-29      38320700.0   
       2021-06-30     2021-07-29      38320700.0   
       2021-09-30     2021-11-02      38285500.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Report Date                                                    
A      2016-10-31                                      2.289000e+09   
       2017-01-31                                      2.241000e+09   
       2017-04-30                                      2.389000e+09   
       2017-07-31                                      2.563000e+09   
       2017-10-31                                      2.678000e+09   
...                                                             ...   
ZYXI   2020-09-30                                      4.122900e+07   
       2020-12-31                                      3.917300e+07   
       2021-03-31                                      3.343900e+07   
       2021-06-30                                      3.233000e+07   
       2021-09-30                                      3.536800e+07   

                    Accounts & Notes Receivable  Inventories  \
Ticker Report Date                                             
A      2016-10-31                   631000000.0  533000000.0   
       2017-01-31                   653000000.0  551000000.0   
       2017-04-30                   677000000.0  548000000.0   
       2017-07-31                   678000000.0  566000000.0   
       2017-10-31                   724000000.0  575000000.0   
...                                         ...          ...   
ZYXI   2020-09-30                     9365000.0    5898000.0   
       2020-12-31                    13837000.0    8635000.0   
       2021-03-31                    14874000.0   10957000.0   
       2021-06-30                    18310000.0    9951000.0   
       2021-09-30                    24234000.0    9154000.0   

                    Total Current Assets  ...  Short Term Debt  \
Ticker Report Date                        ...                    
A      2016-10-31           3.635000e+09  ...              0.0   
       2017-01-31           3.635000e+09  ...      190000000.0   
       2017-04-30           3.800000e+09  ...      241000000.0   
       2017-07-31           3.996000e+09  ...      280000000.0   
       2017-10-31           4.169000e+09  ...      210000000.0   
...                                  ...  ...              ...   
ZYXI   2020-09-30           5.755300e+07  ...              0.0   
       2020-12-31           6.302300e+07  ...          

In [12]:
# add id column
df_balance.insert(0, 'id', range(1, 1 + len(df_balance)))

In [13]:
df_balance

id  SimFinId  Fiscal Year Publish Date Restated Date  \
Ticker Report Date                                                            
A      2016-10-31       1     45846         2016   2016-12-20    2017-12-21   
       2017-10-31       2     45846         2017   2017-12-21    2018-12-20   
       2018-10-31       3     45846         2018   2018-12-20    2018-12-20   
       2019-10-31       4     45846         2019   2019-12-19    2020-12-18   
       2020-10-31       5     45846         2020   2020-12-18    2021-12-17   
...                   ...       ...          ...          ...           ...   
ZYXI   2016-12-31   11204    171401         2016   2017-04-18    2018-02-28   
       2017-12-31   11205    171401         2017   2018-02-28    2019-02-26   
       2018-12-31   11206    171401         2018   2019-02-26    2019-04-30   
       2019-12-31   11207    171401         2019   2020-02-27    2021-02-25   
       2020-12-31   11208    171401         2020   2021-02-25    2022-03-22   

                    Shares (Basic)  \
Ticker Report Date                   
A      2016-10-31      326000000.0   
       2017-10-31      322000000.0   
       2018-10-31      321000000.0   
       2019-10-31      314000000.0   
       2020-10-31      309000000.0   
...                            ...   
ZYXI   2016-12-31       34398357.0   
       2017-12-31       35371600.0   
       2018-12-31       35753300.0   
       2019-12-31       35682900.0   
       2020-12-31       37255900.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Report Date                                                    
A      2016-10-31                                      2.289000e+09   
       2017-10-31                                      2.678000e+09   
       2018-10-31                                      2.247000e+09   
       2019-10-31                                      1.382000e+09   
       2020-10-31                                      1.441000e+09   
...                                                             ...   
ZYXI   2016-12-31                                      2.470000e+05   
       2017-12-31                                      5.565000e+06   
       2018-12-31                                      1.012800e+07   
       2019-12-31                                      1.404000e+07   
       2020-12-31                                      3.917300e+07   

                    Accounts & Notes Receivable  Inventories  \
Ticker Report Date                                             
A      2016-10-31                  6.310000e+08  533000000.0   
       2017-10-31                  7.240000e+08  575000000.0   
       2018-10-31                  7.760000e+08  638000000.0   
       2019-10-31                  9.300000e+08  679000000.0   
       2020-10-31                  1.038000e+09  720000000.0   
...                                         ...          ...   
ZYXI   2016-12-31                  3.028000e+06     107000.0   
       2017-12-31                  2.185000e+06     423000.0   
       2018-12-31                  2.791000e+06     837000.0   
       2019-12-31                  5.833000e+06    2378000.0   
       2020-12-31                  1.383700e+07    8635000.0   

                    Total Current Assets  ...  Short Term Debt  \
Ticker Report Date                        ...                    
A      2016-10-31           3.635000e+09  ...              0.0   
       2017-10-31           4.169000e+09  ...      210000000.0   
       2018-10-31           3.848000e+09  ...              0.0   
       2019-10-31           3.189000e+09  ...      616000000.0   
       2020-10-31           3.415000e+09  ...       75000000.0   
...                                  ...  ...              ...   
ZYXI   2016-12-31           3.422000e+06  ...         118000.0   
       2017-12-31           8.371000e+06  ...         354000.0   
       2018-12-31           1.432400e+07  ...              0.0   
       2019-12-31

In [10]:
# clean columns names
df_balance.columns = [x.lower().replace(" ", "_").replace("?", "").replace("-", "_") \
    .replace(r"/","_").replace("\\","_").replace("%","_per").replace(")","") \
         .replace(r"(","").replace("$","").replace(":", "").replace(",","") for x in df_balance.columns]
df_balance.columns

Index(['simfinid', 'fiscal_year', 'fiscal_period', 'publish_date',
       'restated_date', 'shares_basic',
       'cash_cash_equivalents_&_short_term_investments',
       'accounts_&_notes_receivable', 'inventories', 'total_current_assets',
       'property_plant_&_equipment_net', 'long_term_investments_&_receivables',
       'other_long_term_assets', 'total_noncurrent_assets', 'total_assets',
       'payables_&_accruals', 'short_term_debt', 'total_current_liabilities',
       'long_term_debt', 'total_noncurrent_liabilities', 'total_liabilities',
       'share_capital_&_additional_paid_in_capital', 'treasury_stock',
       'retained_earnings', 'total_equity', 'total_liabilities_&_equity'],
      dtype='object')

In [11]:
#save to csv
df_balance.to_csv('csv/balance_sheet.csv', index=True, sep=';', encoding='utf-8', float_format='%.0f')
